In [1]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/modules")
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
#########################################

computer =  blogin1
working directory set to /home/hbkoziel/awi-models


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import colormaps as cmaps
from matplotlib.colors import ListedColormap
import pyfesom as pf

In [3]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1980
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = True
# load the given biological tracer #
var_id1, var_id2 = 'tr06', 'tr15'
mld_id = 'mixlay'
# export CSV
export_csv = True
########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/CSV/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [4]:
mesh = fesom_mesh(meshpath, get3d = get3d)
#mesh = load_mesh(meshpath, usepickle=True,get3d=True)
mesh.n32 = mesh.n32-1
mesh.zlevs = -mesh.zlevs # depth is made negative

Load 2d part of the grid in 4 second(s)
Load 3d part of the grid in 43 seconds


In [5]:
mesh


FESOM mesh:
path                  = /home/hbkoziel/awi-models/data/mesh/meshArc4.5
alpha, beta, gamma    = 50, 15, -90
number of 2d nodes    = 753179
number of 2d elements = 1480268
number of 3d nodes    = 16950142

        

In [6]:
mesh.zlevs

array([    0.,   -10.,   -20.,   -30.,   -40.,   -50.,   -60.,   -70.,
         -80.,   -90.,  -100.,  -115.,  -135.,  -160.,  -190.,  -230.,
        -280.,  -340.,  -410.,  -490.,  -580.,  -680.,  -790.,  -910.,
       -1040., -1180., -1330., -1500., -1700., -1920., -2150., -2400.,
       -2650., -2900., -3150., -3400., -3650., -3900., -4150., -4400.,
       -4650., -4900., -5150., -5400., -5650., -5900.])

In [ ]:
CHL  = np.zeros((len(years),len(mesh.x2)))
for mo in range(3,9):
    print mo
    for ye in range(0,len(years)):
     print years[ye]
     ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
     f1       = Dataset(ncfile, 'r')  
     
     ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.diag.nc'
     f2       = Dataset(ncfile, 'r')  
        
     if years[ye] < 2000:
        chl1     = f1.variables[var_id1][mo,:]
        chl2     = f1.variables[var_id2][mo,:]
        chl=chl1+chl2
        
        mld     = f2.variables[mld_id][mo,:]
        
     else:
        
        if mo ==3: 
            dayind = (range(45,60))
            month='APR'
        elif mo ==4: 
            dayind = (range(60,75))
            month='MAY'
        elif mo ==5: 
            dayind = (range(75,90))
            month='JUN'
        elif mo ==6: 
            dayind = (range(90,105))
            month='JUL'
        elif mo ==7:
            dayind = (range(105,120))
            month='AUG'
        elif mo ==8: 
            dayind = (range(120,135))
            month='SEP'
            
        chl1     = f1.variables[var_id1][dayind,:].mean(axis=0)
        chl2     = f1.variables[var_id2][dayind,:].mean(axis=0)
        chl=chl1+chl2
         
        mld     = f2.variables[mld_id][dayind,:].mean(axis=0)
        
        
     depth_list = [    0.,   -10.,   -20.,   -30.,   -40.,   -50.,   -60.,   -70.,
         -80.,   -90.,  -100.,  -115.,  -135.,  -160.,  -190.,  -230.,
        -280.,  -340.,  -410.,  -490.]
     level_data  = np.zeros((len(mesh.x2),len(depth_list)))
     
     for i in range(len(depth_list)):

        #print 'Load data at '+str(depth_list[i])+'m'
        level_data1, elem_no_nan1 = pf.get_data(chl,mesh,depth_list[i])
        mask = np.ones(size(mld))
        ind_mld = np.argwhere(mld <= depth_list[i])
        if len(ind_mld)>0:
            mask[ind_mld] = np.zeros
            level_data1[mask==0]=np.zeros
        level_data1=np.nan_to_num(level_data1)
        #nan_array = np.isnan(level_data1)
        #level_data1[nan_array]=0
        level_data[:,i]  = level_data1
        
     data_int=np.trapz(-level_data,depth_list,axis=1)
     CHL[ye,:]=data_int
    data2=CHL.mean(axis=0)
    print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
    print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
    print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
    print 'Max and min: ',np.max(data2),np.min(data2)
    
    
    result_fit = np.ma.polyfit(years,CHL,1)
    trend = result_fit[0,:]
    
    if export_csv == True:
        np.savetxt(outputpath+'CHLmld_'+month+'_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', CHL, delimiter=";")
        np.savetxt(outputpath+'CHLmld_'+month+'_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
        np.savetxt(outputpath+'CHLmld_'+month+'_'+str(first_year)+'_'+str(last_year)+'_trend.csv', trend, delimiter=";")
        print 'exporting done ...'

3
1980
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
1981
For depth 0.0 model level 0.0 wi

For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
1989
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 wil

For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
1998
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will 

For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
2006
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 wil

For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
2015
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will 

...

For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  15.946954152
Max and min:  97.5220455974 0.000500000027791
exporting done ...
8
1980
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model lev

For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
1988
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 wil

For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
1997
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will 

For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
2005
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 wil

For depth -70.0 model level -70.0 will be used
For depth -80.0 model level -80.0 will be used
For depth -90.0 model level -90.0 will be used
For depth -100.0 model level -100.0 will be used
For depth -115.0 model level -115.0 will be used
For depth -135.0 model level -135.0 will be used
For depth -160.0 model level -160.0 will be used
For depth -190.0 model level -190.0 will be used
For depth -230.0 model level -230.0 will be used
For depth -280.0 model level -280.0 will be used
For depth -340.0 model level -340.0 will be used
For depth -410.0 model level -410.0 will be used
For depth -490.0 model level -490.0 will be used
2014
For depth 0.0 model level 0.0 will be used
For depth -10.0 model level -10.0 will be used
For depth -20.0 model level -20.0 will be used
For depth -30.0 model level -30.0 will be used
For depth -40.0 model level -40.0 will be used
For depth -50.0 model level -50.0 will be used
For depth -60.0 model level -60.0 will be used
For depth -70.0 model level -70.0 will 